# INELASTIC BRUSSELS NETWORK STA

This file first shows how to run an inelastic Static Traffic Assignment in dyntapy on a larger network. It also shows examples of how to run an inelastic assignment with tolls.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import sys
sys.path.append("../../..")
import pandas as pd
import geopandas as gpd
import numpy as np
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from dyntapy.toll import create_toll_object
from pyproj import Proj, transform
from pickle import dump

2. In this case we would like to load a specific network. To make the path a bit more flexible, the buffer parameter which is the radius of the zoning file and the city are parameters that can be adapted easily to call a specific file path. Here a radius of 40km and the city of Brussels are used. These steps are executed by fully running the next block of code. 

In [ ]:
# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
buffer = 40
city = 'BRUSSEL'

# load in the paths to the OD data and the zoning file
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'STA'
zoning_path = data_path + os.path.sep + 'shapefiles' + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
od_path = data_path + os.path.sep + 'OD_matrix' + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"
print(zoning_path)

3. The next steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically
- Create or load an OD matrix (with the demand) onto the centroids

  One of the important steps that you should not forget is projecting everything in the correct coordinate reference system (CRS). Especially when zones are aggregated using QGIS, this is something that you should be aware of. 

In [ ]:
# Load the network for Brussels. 
network = road_network_from_place(city, buffer_dist_close=buffer*1000)
network = relabel_graph(network)
show_network(network,notebook=True)

# Read the zoning and OD file with the specified path
zoning = gpd.read_file(zoning_path)
od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required.
network = add_centroids(network, x_centroids,y_centroids,k=1, method='link')
network = relabel_graph(network)
# show_network(network, notebook=True)

# Create OD graph
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 

4. Now all the elements are there to run an STA in the next block of code. When this assignment is performed, there is now a loaded network visible with the traffic flows as an additional characteristic. 

In [ ]:
# The network, zoning file and od matrix have been correctly loaded and linked by now. 
# This means that we can now run an assignment on the real demand! 
toll_value = 0.05
toll_ids = [1490]
toll_method = 'single'
toll = create_toll_object(network, toll_method, toll_ids, toll_value)

assignment = StaticAssignment(network,od_graph,toll)
result = assignment.run('dial_b')
show_network(network, flows = result.flows, show_nodes=False)

    Congratulations, you have now been able to run an STA on a tolled, scaled-up network!

---------------------------------------------------------------------------

5. The last step is preparing for the Heeds iterations. Save the full network (containing roads, centroids and connectors) with the OD matrix to avoid reloading these files each iteration. Because both remain static in an inelastic use-case, they are saved with a given path.

    If you do not want to run a Heeds optimisation, you can also use (and alter) the objective function written in the code below to evaluate the effect of different toll values on the network.

In [ ]:
buffer = str(buffer)
HEEDS_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'HEEDS_input'
network_path = HEEDS_path + os.path.sep + 'network_with_centroids' + os.path.sep + 'inelastic_' + city + '_' + buffer
od_graph_path = HEEDS_path + os.path.sep + 'od_graph' + os.path.sep + 'inelastic_' + city + '_' + buffer

with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')

with open(od_graph_path, 'wb') as f:
    dump(od_graph,f)
    print(f'od_graph saved at f{od_graph_path}')


# objective function to determine what effect the toll has on the network
veh_hours = result.link_costs * result.flows
objective = sum(veh_hours)
print("Objective: %s" % objective)

    Congratulations, you are now ready to go to the next step: HEEDS!